In [13]:
!pip install scikit-surprise

In [14]:

# Import required libraries:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [15]:

!pip install joblib>=1.0.0

In [16]:

# Load the data from the MovieLens dataset
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [17]:
# Create a Surprise Dataset
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

In [18]:
# Build a user-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'user_based': True,
}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model's performance
rmse = accuracy.rmse(predictions)

RMSE: nan


In [20]:
# Function to recommend movies for a given user
def recommend_movies(user_id, n=10):
    movies_watched = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    unrated_movies = movies[~movies['movieId'].isin(movies_watched)]

    recommendations = []
    for movie_id in unrated_movies['movieId']:
        predicted_rating = model.predict(user_id, movie_id).est
        recommendations.append((movie_id, predicted_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_n_recommendations = recommendations[:n]

    recommended_movies = movies[movies['movieId'].isin([movie[0] for movie in top_n_recommendations])]
    return recommended_movies[['movieId', 'title']]

In [21]:
# Example_1:
user_id = 1
recommended_movies = recommend_movies(user_id, n=10)
print(f"Top 10 recommended movies for User {user_id}:")
print(recommended_movies)

Top 10 recommended movies for User 1:
      movieId                                             title
380       385                    Man of No Importance, A (1994)
385       390               Faster Pussycat! Kill! Kill! (1965)
483       488                               M. Butterfly (1993)
613       621                         My Favorite Season (1993)
690       705                                       Cosi (1996)
943       964                       Angel and the Badman (1947)
952       973                              Meet John Doe (1941)
1026     1050                        Looking for Richard (1996)
1295     1329  Blood for Dracula (Andy Warhol's Dracula) (1974)
1421     1463                           That Old Feeling (1997)


In [22]:
# Example_2:
user_id = 5
recommended_movies = recommend_movies(user_id, n=10)
print(f"Top 10 recommended movies for User {user_id}:")
print(recommended_movies)

Top 10 recommended movies for User 5:
      movieId                                title
380       385       Man of No Importance, A (1994)
385       390  Faster Pussycat! Kill! Kill! (1965)
483       488                  M. Butterfly (1993)
610       618                      Two Much (1995)
613       621            My Favorite Season (1993)
640       651                Superweib, Das (1996)
690       705                          Cosi (1996)
943       964          Angel and the Badman (1947)
952       973                 Meet John Doe (1941)
1026     1050           Looking for Richard (1996)


In [23]:
# Example_3:
user_id = 10
recommended_movies = recommend_movies(user_id, n=5)
print(f"Top 5 recommended movies for User {user_id}:")
print(recommended_movies)

Top 5 recommended movies for User 10:
     movieId                                title
380      385       Man of No Importance, A (1994)
385      390  Faster Pussycat! Kill! Kill! (1965)
483      488                  M. Butterfly (1993)
574      582        Métisse (Café au Lait) (1993)
613      621            My Favorite Season (1993)
